<div class="alert alert-warning">
Решение 0_input_
</div>

In [1]:
import sys
import typing as tp


def input_(prompt: tp.Optional[str] = None,
           inp: tp.Optional[tp.IO[str]] = None,
           out: tp.Optional[tp.IO[str]] = None) -> tp.Optional[str]:
    """Read a string from `inp` stream. The trailing newline is stripped.

    The `prompt` string, if given, is printed to `out` stream without a
    trailing newline before reading input.

    If the user hits EOF (*nix: Ctrl-D, Windows: Ctrl-Z+Return), return None.

    `inp` and `out` arguments are optional and should default to `sys.stdin`
    and `sys.stdout` respectively.
    """
    if inp is None:
        inp = sys.stdin
    if out is None:
        out = sys.stdout
    if prompt is not None:
        out.write(prompt)
    out.flush()
    s = inp.readline()
    if s == '':
        return None
    return s.rstrip('\n')


def illegal_input_(prompt, inp, out):  # type: ignore
    """Just to check that forbidden_function decorator works"""
    sys.stdin = inp
    sys.stdout = out
    return input(prompt)


<div class="alert alert-warning">
Решение 1_calc
</div>

In [2]:
import math
from typing import Any, Dict, Optional

PROMPT = '>>> '


def run_calc(context: Optional[Dict[str, Any]] = None) -> None:
    """Run interactive calculator session in specified namespace"""
    if context is None:
        context = {}
    context['__builtins__'] = {}
    while True:
        try:
            expr = input(PROMPT)
        except EOFError:
            print()
            break
        print(eval(expr, context))


if __name__ == '__main__':
    context = {'math': math}
    run_calc(context)


>>> 2+2
4
>>> 


SyntaxError: unexpected EOF while parsing (<string>, line 0)

<div class="alert alert-warning">
Решение 2_byteme
</div>

In [3]:
# type: ignore
# flake8: noqa


def f0():
    pass


def f1():
    a = 0
    return a


def f2():
    a = 0
    print(a)


def f3():
    a = 0
    a += 1
    print(a)


def f4():
    return range(10)


def f5():
    for i in range(10):
        print(i)


def f6():
    a = 0
    for i in range(10):
        a += 1
    print(a)


def f7():
    return [1, *(2, 3), 4]


def f8():
    x, y = 1, 2


def f9():
    return 1 if 1 == 1 else 2


def f10():
    for i in range(10):
        if i == 3:
            break


def f11():
    list_ = [1, 2, 3]
    dict_ = {'a': 1, 'b': 2}
    return list_, dict_


def f12():
    a = 1
    b = 2
    c = 3
    d = 4
    e = 5
    return a + b * c / d ** e


<div class="alert alert-warning">
Решение 3_normalize_path
</div>

In [4]:
from typing import List


def normalize_path(path: str) -> str:
    """
    :param path: unix path to normalize
    :return: normalized path
    """
    stack: List[str] = []
    absolute = path.startswith('/')
    parts = path.split('/')
    for part in parts:
        if not part:
            # ...//... -> .../...
            continue
        elif part == '.':
            # .../abc/. -> .../abc
            # /. -> /
            # ./... -> ...
            # . -> '' (see below)
            continue
        elif part == '..':
            if stack:
                if stack[-1] == '..':
                    # ../.. -> ../..
                    stack.append('..')
                else:
                    # .../abc/.. -> ...
                    stack.pop()
            else:
                if absolute:
                    # /.. -> /
                    continue
                else:
                    # .. -> ..
                    stack.append('..')
        else:
            stack.append(part)
    if absolute:
        return '/' + '/'.join(stack)
    else:
        # abc/.. -> .
        return '/'.join(stack) or '.'


<div class="alert alert-warning">
Решение 4_profiler
</div>

In [6]:
import functools
from datetime import datetime


def profiler(func):  # type: ignore
    """
    Returns profiling decorator, which counts calls of function
    and measure last function execution time.
    Results are stored as function attributes: `calls`, `last_time_taken`
    :param func: function to decorate
    :return: decorator, which wraps any function passed
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):  # type: ignore
        outer_call = not wrapper._in_recursion
        if outer_call:
            wrapper.calls = 0
            wrapper._in_recursion = True

        wrapper.calls += 1

        start = datetime.now()
        result = func(*args, **kwargs)
        delta = datetime.now() - start

        wrapper.last_time_taken = delta.total_seconds()

        if outer_call:
            wrapper._in_recursion = False
        return result

    wrapper._in_recursion = False
    return wrapper


<div class="alert alert-warning">
Решение 5_lru_cache
</div>

In [7]:
import functools
from collections import OrderedDict
from typing import Callable, Any, Tuple, TypeVar, cast


Function = TypeVar('Function', bound=Callable[..., Any])


def cache(max_size: int) -> Callable[[Function], Function]:
    """
    Returns decorator, which stores result of function
    for `max_size` most recent function arguments.
    :param max_size: max amount of unique arguments to store values for
    :return: decorator, which wraps any function passed
    """
    def _cache(func: Function) -> Function:
        storage: OrderedDict[Tuple[Any], Any] = OrderedDict()
        sentinel = object()

        @functools.wraps(func)
        def wrapper(*args, **kwargs):  # type: ignore
            storage_key = (
                args,
                tuple((key, kwargs[key]) for key in sorted(kwargs))
            )

            value = storage.get(storage_key, sentinel)
            if value is not sentinel:
                storage.move_to_end(storage_key, last=True)
                return value

            value = func(*args, **kwargs)
            storage[storage_key] = value
            if len(storage) > max_size:
                storage.popitem(last=False)
            return value

        return cast(Function, wrapper)
    return _cache
